In [3]:
import pandas as pd
import random

df = pd.read_csv('beers.csv')
ounce_to_ml = 29.574
sample_size = 300

beer_styles = df['style']
style_abundance = (beer_styles.value_counts(normalize=True) * 100).to_dict()


beers = df[['name', 'abv', 'ibu', 'ounces', 'style']].dropna()
beers['abv'] = round(100 * beers['abv'], 2)
beers['ounces'] *= ounce_to_ml
beers['ounces'] = beers['ounces'].astype('int')
beers['style'] = (beers['style'].apply(lambda x: style_abundance[x])).astype('float')
beers.columns = ['nazwa', 'procenty', 'ibu', 'ilosc ml', 'pospolitosc']
beers.reset_index(drop=True, inplace=True)
beers_sample = beers.sample(sample_size).reset_index(drop=True)
beers_sample['cena'] = [round(random.uniform(8, 20), 2) for _ in range(sample_size)]
beers_sample.head(10)

,nazwa,procenty,ibu,ilosc ml,pospolitosc,cena
0,Wild Wolf Wee Heavy Scottish Style Ale,5.7,20.0,354,0.623701,18.88
1,Ale Cider,6.5,8.0,473,2.037422,14.13
2,Dog Days Summer Ale,4.5,28.0,354,1.746362,13.49
3,Fortunate Islands,4.7,46.0,473,4.033264,12.07
4,Overlord Imperial IPA,8.5,115.0,473,4.365904,9.49
5,Ultra Gnar Gnar IPA,6.7,60.0,354,17.629938,14.87
6,Point Nude Beach Summer Wheat (2010),5.0,7.0,354,4.033264,14.48
7,East India Pale Ale,6.8,47.0,473,0.540541,10.10
8,Sly Fox Christmas Ale 2012 (2012),5.5,16.0,354,0.623701,14.93
9,Na Zdraví Pilsner,4.8,32.0,473,1.164241,13.95


In [4]:
beers_sample.to_csv('piwa_kraftowe.csv')